In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
asakusaline = pd.read_csv('data/metro_data/asakusaline.csv')
fukutoshineline = pd.read_csv('data/metro_data/fukutoshinline.csv')
gbline = pd.read_csv('data/metro_data/gbline.csv')
hankyukyotomainline = pd.read_csv('data/metro_data/hankyukyotomainline.csv')
keikyumainline = pd.read_csv('data/metro_data/keikyumainline.csv', encoding = 'cp949')
newyork = pd.read_csv('data/metro_data/newyork.csv')
newyorkline6 = pd.read_csv('data/metro_data/newyorkline6.csv', encoding = 'cp949')
seoulline1 = pd.read_csv('data/metro_data/seoulline1.csv', encoding = 'cp949')
seoulline4 = pd.read_csv('data/metro_data/seoulline4.csv', encoding = 'cp949')
seoulline9 = pd.read_csv('data/metro_data/seoulline9.csv', encoding = 'cp949')
shinjukuline = pd.read_csv('data/metro_data/shinjukuline.csv', encoding = 'cp949')
tokyudenentoshiline= pd.read_csv('data/metro_data/tokyudenentoshiline.csv', encoding = 'cp949')
tokyutoyokoline = pd.read_csv('data/metro_data/tokyutoyokoline.csv', encoding = 'cp949')
tozailine = pd.read_csv('data/metro_data/tozailine.csv', encoding = 'cp949')

## 데이터 처리

DEA 괜찮은 애들로만 학습 (DEA 안좋으면 문제있는 노선인데 그걸로 학습시키면 안되지않을까..?)

In [ ]:
hankyukyotomainline['km'] += 2.4

In [ ]:
def change_people(data): # change str to int
  for i in range(len(data)):
    a = data['people'][i]
    if type(a) == str and ',' in a:
      a = a.replace(',', '')
      data['people'][i] = int(a)
    elif type(a) == str:
      a = int(a)
      data['people'][i] = a
    elif type(a) == int:
      pass


In [ ]:
def time_to_num(a): # change time to float
  if ':' in a:
    idx = a.index(':')
    m = int(a[:idx])
    if int(a[idx + 1:]) == 0:
      s = 0
    else:
      s = 0.5
    time = m+s
    return time
  else:
    return a

In [ ]:
def change_km(data): # accumulated distance to distance gap
  data_copy = data.copy()
  for i in range(len(data)):
    if i > 0:
      data['km'][i] = round(data_copy['km'][i] - data_copy['km'][i-1],1)

In [ ]:
# lines with good DEA, bad ones are not partcipating in model training
data_list = [asakusaline, hankyukyotomainline, shinjukuline, tokyudenentoshiline, tokyutoyokoline] 

for set in data_list:
  change_km(set)
  change_people(set)
  for i in range(len(set)):
    a = time_to_num(set['time'][i])
    set['time'][i] = a


<ipython-input-8-80242929214c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['km'][i] = round(data_copy['km'][i] - data_copy['km'][i-1],1)
<ipython-input-8-80242929214c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['km'][i] = round(data_copy['km'][i] - data_copy['km'][i-1],1)
<ipython-input-8-80242929214c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['km'][i] = round(data_copy['km'][i] - data_copy['km'][i-1],1

In [ ]:
df = pd.concat([asakusaline, hankyukyotomainline, shinjukuline, tokyudenentoshiline, tokyutoyokoline], axis = 0)

In [ ]:
columns_to_drop = ['Unnamed: 0', 'statnNo', 'location', 'trainTrans', 'airTrans', 'platform']
use_df = df.drop(columns = columns_to_drop, inplace = False) # reset the index (must be done)
use_df = use_df.reset_index()
del use_df['index']

In [ ]:
# Deleting Outliers

q3 = use_df['people'].quantile(0.75)
q1 = use_df['people'].quantile(0.25)
iqr = q3 - q1
def outlier_ch(data):
  people_num = data['people']
  if people_num > (q3 + 1.5*iqr) or people_num <( q1 -1.5*iqr):
    return True
  else:
    return False
use_df['outlier'] = use_df.apply(outlier_ch, axis = 1)

use_df = use_df.loc[use_df['outlier'] == False]
del use_df['outlier']
use_df

## 급행 아닌 경우의 이용량 예측




In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# station_name	length	time	express	transfer	sameTrans	diffTrans	transport	store	public_med_facility	multi_facility	/ people

In [ ]:
not_express_dataset = use_df[use_df['express'] == 0] #express 0
not_express_dataset.head(2)

,statnNm,km,express,people,time,transfer,sameTrans,diffTrans,transport,store,public_med_facility,multi_facility
4,Higashi-ginza,0.9,0,83607,3,3,0,3,0,46,5,8
5,Takaracho,0.8,0,30176,1,0,0,0,0,28,2,15


In [ ]:
express_dataset = use_df[use_df['express'] == 1] #express 1

In [ ]:
# express estimate (일반역일 땐 not_express_dataset)
target_data = express_dataset['people'].to_numpy()
input_data = (express_dataset.drop(columns = ['statnNm','people'])).to_numpy()

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!python -m pip install autogluon

  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.20.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train = TabularDataset(express_dataset.drop(['statnNm'], axis=1))

In [ ]:
predictor = TabularPredictor(label='people', eval_metric='rmse').fit(train_data = train, verbosity = 2, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240206_071805"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240206_071805/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 915 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 2685 seconds.
Starting full fit now with num_stack_le

In [ ]:
predictor.fit_summary()
board = predictor.leaderboard(train, silent=True)
board

*** Summary of fit() ***
Estimated performance of each model:
                           model     score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            WeightedEnsemble_L2 -25326.209691  root_mean_squared_error       0.638123  143.454301                0.000846           1.174105            2       True        109
1    NeuralNetFastAI_r103_BAG_L1 -26994.947713  root_mean_squared_error       0.127291   30.613374                0.127291          30.613374            1       True         36
2     NeuralNetFastAI_r88_BAG_L1 -27633.596854  root_mean_squared_error       0.135243   30.653811                0.135243          30.653811            1       True         66
3      NeuralNetFastAI_r4_BAG_L1 -27673.910742  root_mean_squared_error       0.151641   30.315017                0.151641          30.315017            1       True         96
4     NeuralNetTorch_r135_BAG_L1 -29522.275279  root_

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-0.000000,-37511.237974,root_mean_squared_error,0.018778,0.014212,0.006318,0.018778,0.014212,0.006318,1,True,2
1,RandomForest_r166_BAG_L1,-11610.314000,-32728.376094,root_mean_squared_error,0.156223,0.195881,0.673643,0.156223,0.195881,0.673643,1,True,74
2,RandomForest_r195_BAG_L1,-11651.540012,-32920.091930,root_mean_squared_error,0.070646,0.167461,0.632565,0.070646,0.167461,0.632565,1,True,24
3,ExtraTrees_r197_BAG_L1,-11695.089692,-33078.731489,root_mean_squared_error,0.067212,0.156401,0.742726,0.067212,0.156401,0.742726,1,True,89
4,ExtraTreesMSE_BAG_L1,-11695.089692,-33078.731489,root_mean_squared_error,0.080413,0.170240,0.581856,0.080413,0.170240,0.581856,1,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,LightGBM_r135_BAG_L1,-37881.352911,-39201.105232,root_mean_squared_error,0.026606,0.035502,16.386253,0.026606,0.035502,16.386253,1,True,80
105,LightGBM_r96_BAG_L1,-37881.352911,-39201.105232,root_mean_squared_error,0.031808,0.028891,15.794959,0.031808,0.028891,15.794959,1,True,17
106,LightGBMXT_BAG_L1,-37881.352911,-39201.105232,root_mean_squared_error,0.036904,0.069264,16.762496,0.036904,0.069264,16.762496,1,True,3
107,RandomForest_r34_BAG_L1,-37883.140927,-38813.723861,root_mean_squared_error,0.068046,0.199100,0.604938,0.068046,0.199100,0.604938,1,True,58


In [ ]:
predictor.feature_importance(data=train)

These features in provided data are not utilized by the predictor and will be ignored: ['express']
Computing feature importance via permutation shuffling for 9 features using 46 rows with 5 shuffle sets...
	39.87s	= Expected runtime (7.97s per shuffle set)
	7.56s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
public_med_facility,5292.878309,990.630858,0.000141,5,7332.600937,3253.155680
km,4958.838357,985.063488,0.000177,5,6987.097693,2930.579021
store,4692.694062,667.130941,0.000048,5,6066.325891,3319.062233
diffTrans,4336.496039,724.368493,0.000090,5,5827.980779,2845.011300
multi_facility,3319.892729,755.197259,0.000300,5,4874.854323,1764.931134
transfer,2844.283987,466.439814,0.000084,5,3804.690011,1883.877963
sameTrans,2726.185093,911.260194,0.001299,5,4602.482426,849.887759
transport,2173.805427,479.642578,0.000267,5,3161.396125,1186.214730
time,762.918600,394.125901,0.006183,5,1574.429278,-48.592078


In [ ]:
predictor.evaluate(train)

{'root_mean_squared_error': -21141.68165768975,
 'mean_squared_error': -446970703.3150953,
 'mean_absolute_error': -17359.652513586956,
 'r2': 0.6885212425330132,
 'pearsonr': 0.8656037072819698,
 'median_absolute_error': -14575.505859375}

In [ ]:
best_model = predictor.get_model_best()

`get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!


In [ ]:
# Begin with n2ex predction, and then ex2n prediction

# change the station DEA input each time you work on
before = pd.read_excel('result/before_DEA/newyork_before_DEA.xlsx')
after = pd.read_excel('result/after_DEA/newyork_after_DEA.xlsx')
n2ex = before[before['express'] == 0].merge(after[after['express']==1], on = 'statnNm') #normal to express

n2ex

In [ ]:
input_data = n2ex[['km_x', 'time_x', 'transfer_x', 'sameTrans_x', 'diffTrans_x', 'transport_x', 'store_x', 'public_med_facility_x', 'multi_facility_x']]
for i in range(9):
  a = input_data.columns[i]
  input_data = input_data.rename(columns={a:a[:-2]})

In [ ]:
pred = predictor.predict(input_data, model=best_model)
pred

In [ ]:
before = pd.read_excel('/content/drive/MyDrive/URP (Metro)/Week 7/DEA/before_DEA/gbline_before_DEA.xlsx')
after = pd.read_excel('/content/drive/MyDrive/URP (Metro)/Week 7/DEA/after_DEA/gbline_after_DEA.xlsx')
ex2n = pd.merge(before[before['express'] == 1], after[after['express']==0], left_on='statnNm', right_on='statnNm', how='inner') #express to normal
ex2n

In [ ]:
input_data = ex2n[['km_x', 'time_x', 'transfer_x', 'sameTrans_x', 'diffTrans_x', 'transport_x', 'store_x', 'public_med_facility_x', 'multi_facility_x']]
for i in range(9):
  a = input_data.columns[i]
  input_data = input_data.rename(columns={a:a[:-2]})

In [ ]:
pred = predictor.predict(input_data, model=best_model)
pred